# 13: Classic Newton-Raphson

This week, we look at a basic algorithm for solving problems with a computer, just for some last practice in solid programming skills.


## Newton-Raphson


Newton's method (or Newton-Raphson) is a basic tool to find the zeroes (or at least one zero) of an equation:

$$
x\, | \,f(x) = 0
$$

This week we will use it to find the square root of a number, so the equation is:

$$
x^2 - a = 0
$$

for $x$ to be determined, $a$ is the number that we need to find the square root of.

For the square:

$f'(x)=2x$

Newton's method is a simple iteration, applied until some small value for $\epsilon$ is achieved:

$$
x \leftarrow x - \frac{f(x)}{f'(x)}
$$

$$
\epsilon = ||f(x) - 0||
$$

Hopefully this is intuitive: if the gradient of $f$ is small, then we need to take big steps and *vice versa*.  If the evaluated size of $f$ is big in relation to the gradient; likewise big steps seem reasonable.



## Argument for convergence

This is an argument for convergence, not a proof, because in fact many pathological situations can be constructed in which the above algorithm does not converge. On the wikipedia page, there is a very attractive fractal plot showing decoratively the areas of the complex plane in which Newton-Raphson will run into trouble when trying to solve $x^5-1=0$ [ https://en.wikipedia.org/wiki/Newton%27s_method#/media/File:Newtroot_1_0_0_0_0_m1.png ].


With this in mind, let's make a Taylor series for the value of $f(x+\alpha)$ for some small $\alpha$ such that we are close to the solution in $x$:

$$
f(x+\alpha) = f(x) + \alpha f'(x) + \ldots
$$

Our challenge here is to estimate what is the $\alpha$, such that we can subtract it from our guess and approach the true $x$.

We just forget about the high-order terms in $\alpha^2 f''(x)/2,\, \alpha^3 f'''(x)/6,\,\ldots$ because $\alpha$ is small and maybe we can assume also that higher-order derivatives are also small relative to lower-order.  For an exponential function high-order derivatives are the same, not smaller, but we should still be safe even then  as the denominator is anyway increasing factorially.

Now divide by $f'$:

$$
\frac{f(x+\alpha)}{f'(x)} = \frac{f(x)}{f'(x)} + \alpha 
$$


$$
\frac{f(x+\alpha) -f(x)}{f'(x)} =  \alpha 
$$

We are trying to solve for a function which has a zero at $f(x)$, so forget that term as well, and our estimate for the error in $x$, $\alpha$, becomes:

$$
\alpha  \approx \frac{f(x+\alpha)}{f'(x)} 
$$

Great, we approximate $f'(x)$ ($x$ is not known yet) as $\approx f'(x+\alpha)$ (because $x+\alpha$ is our current estimate of the ultimate $x$) and we have our update rule.  To review assumptions then:

1)  The first two terms or a Taylor series are a good approximation for $f(x+\alpha)$? Mostly good, fails for some functions, or for many functions if the initial guess is bad.  Excellent for $f(x)=Ax$ (second derivative vanishes), but then of course we don't need an iterative algorithm.

2) $f'(x) \approx f'(x+\alpha)$ : this is the same assumption in disguise, we are saying that the second derivative is small.




## Starting guess

Finding a good initial guess for a Newton-Raphson is crucial.  If you know that the equation has multiple roots, then you will need to make multiple guesses, and be sure that each of them converges to one of the unique roots.

There are some cool tricks for this, consider if we are trying to find a square root:

$$
y=x^2
$$

($x$ unkown, $y$ known).  

$$
\log_2 y = 2 \log_2 x
$$


$$
2^{\frac{\log_2 y}{2}} = x
$$

If we are using floating point (or double) numbers, then we should be able to at least *estimate* logs really easily, and maybe get the exact log without too much trouble.  The reason is the way that floats are stored in a computer: the 32 bits (or 64 for a double) are divided up and signify two separate numbers, $a,c$:

$$
x = \mathbf{a} \times b^\mathbf{c}
$$

Formally, $a,b,c$ are known as the 'significand', 'base' and 'exponent'.  The base, $b$, is 2 in every case that I have seen, so to estimate the $log_2$ of a floating point number we can just extract the value of $c$ directly.



In [9]:
int fast_approx_sqrt_jtb( double x ){
    
    int     exponent;
    double  approx_sqrt, significand;
    
    if( x  <= 0.0 ) return (0.0); //real sqrt is undefined for x<=0, so just return a zero.
    
    
    //frexp() : stores an integer value in *exp such that x \times 2^(*exp) == arg
    significand = frexp( x, &exponent );
    printf("x=%lf , equiv to %lf x 2^(%i)\n", x, significand, exponent);
    
    //formula in cell above
    exponent /= 2; 
    
    //formula above: left-shift operator "<<" moves bits to the left.
    //For integers, x << y is equivalent to x *= 2^y.
    approx_sqrt = (double)( 1 << exponent ); 
    
    return( approx_sqrt );
}

In [10]:
void test_jtb_approx_sqrt(){
    
    double test_values[] = {0.0, 1.0, 2.0, 3.14159, 17.0, 44.0, 64.0, 1e6, 1e12};\
    int    N_test      = 9;
    int    i;
    
    double guess, reference;
    
    for( i = 0; i < N_test; i++ ){
        
        reference = sqrt(test_values[i]);
        guess     = fast_approx_sqrt_jtb( test_values[i] );
        
        printf("%i : x^2=%.3f reference: %.4f guess: %.4f\n\n", i, test_values[i], reference, guess);
        
    }
    
}

test_jtb_approx_sqrt();

0 : x^2=0.000 reference: 0.0000 guess: 0.0000

x=1.000000 , equiv to 0.500000 x 2^(1)
1 : x^2=1.000 reference: 1.0000 guess: 1.0000

x=2.000000 , equiv to 0.500000 x 2^(2)
2 : x^2=2.000 reference: 1.4142 guess: 2.0000

x=3.141590 , equiv to 0.785397 x 2^(2)
3 : x^2=3.142 reference: 1.7725 guess: 2.0000

x=17.000000 , equiv to 0.531250 x 2^(5)
4 : x^2=17.000 reference: 4.1231 guess: 4.0000

x=44.000000 , equiv to 0.687500 x 2^(6)
5 : x^2=44.000 reference: 6.6332 guess: 8.0000

x=64.000000 , equiv to 0.500000 x 2^(7)
6 : x^2=64.000 reference: 8.0000 guess: 8.0000

x=1000000.000000 , equiv to 0.953674 x 2^(20)
7 : x^2=1000000.000 reference: 1000.0000 guess: 1024.0000

x=1000000000000.000000 , equiv to 0.909495 x 2^(40)
8 : x^2=1000000000000.000 reference: 1000000.0000 guess: 1048576.0000



## Carmack fast inverse square root

Presenting the above JTB fast square root, I am obliged to credit the Carmack inverse square root algorithm, which was the first well-documented appearance in public source code of this type of approach, that is of taking the square root quickly using the structure of **float** datatypes to get a fast approximate logarithm.

Below code is a reproduction of original source from the game Quake III arena, with comments included as verbatim:



In [11]:
float Q_rsqrt( float number )
{
	long i;
	float x2, y;
	const float threehalfs = 1.5F;

	x2 = number * 0.5F;
	y  = number;
	i  = * ( long * ) &y;                       // evil floating point bit level hacking
	i  = 0x5f3759df - ( i >> 1 );               // what the fuck? 
	y  = * ( float * ) &i;
	y  = y * ( threehalfs - ( x2 * y * y ) );   // 1st iteration
//	y  = y * ( threehalfs - ( x2 * y * y ) );   // 2nd iteration, this can be removed

	return y;
}

The above source code is totally unreadable, and makes assumptions about the structure of floating point numbers which are not guaranteed to be stable across multiple processor designs, but otherwise it is roughly the same idea.

A single iteration of Newton-Raphson is baked into the function as the final line, from comments it seems that Carmack tried two iterations but was in the end satisfied with the performance after one.

## Assignment

This week I'd like you to write and validate a code to find the cube root of a (positive) number by Newton-Raphson.  Look at my code and explanations for inspiration, not Carmack's, that stuff is just there for completeness and as a historical reference.

You should make a fast initial guess, following the sort of logic that I outlined for my fast square root.

You should implement Newton-Raphson with a tolerance such that the error is less than $1\times 10^{-16}$, and run in a loop over a range of values to prove to me that it is indeed working well.


## Formula for 'quick' inverse cube root

$$
y=x^3
$$

$$
\log_2 y = 3\log_2x
$$

$$
\log_2x = \frac{\log_2y}{3}
$$

$$
x = 2^{\frac{\log_2y}{3}}
$$

$$
f'(x) = 3x^2
$$

$$
x \leftarrow x - \frac{x^3-y}{3x^2} 
$$


In [12]:
double fast_inverse_cube_root( double y){
    double approx_cuberoot, significand;
    int     exponent;
    
    //frexp() : stores an integer value in *exp such that x \times 2^(*exp) == arg
    significand = frexp( y, &exponent );
    
    //formula in cell above
    exponent /= 3; 
    
    //formula above: left-shift operator "<<" moves bits to the left.
    //For integers, x << y is equivalent to x *= 2^y.
    approx_cuberoot = (double)( 1 << exponent ); 
    
    return( approx_cuberoot);  
}

In [35]:
double NR( double y, double z ){ // cube to find the root of and reference value of the root
    double EPSILON = pow(10,-16);
    double x_ini, x_old, x_new, x_delta;
    int iterations = 0;
    
    x_ini = fast_inverse_cube_root( y );
    printf("initial guess: %lf \n", x_ini);
    x_old = x_ini;
    x_delta = std::abs(y-z);
    
    while (x_delta > EPSILON){
        iterations += 1;
        x_new = x_old - (pow(x_old,3) - y)/(3*pow(x_old,2));
        x_delta = std::abs(x_new - z);
        // break to avoid endless loop, change this to get higher precision
        if (iterations > pow(10,7)){ 
            printf("Forcefully exited the loop\n");
            break;
        }
        x_old = x_new;
    }
    printf("true root           :%.16lf\n",z);
    printf("calculated root is  :");
    return(x_new);
}

In [36]:
int test_NR(double y, double z){
    double res;
    
    printf("")
    
    res = NR(y,z);
    
    printf("%.16lf\n\n", res);
    return(EXIT_SUCCESS);
}


In [37]:
test_NR(125,5);
test_NR(27,3);
test_NR(M_PI,1.46459188756152326302);
test_NR(216,6);
test_NR(343,7);
test_NR(421.875,7.5);

initial guess: 4.000000 
true root           :5.0000000000000000
calculated root is  :5.0000000000000000

initial guess: 2.000000 
true root           :3.0000000000000000
calculated root is  :3.0000000000000000

initial guess: 1.000000 
Forcefully exited the loop
true root           :1.4645918875615234
calculated root is  :1.4645918875615231

initial guess: 4.000000 
true root           :6.0000000000000000
calculated root is  :6.0000000000000000

initial guess: 8.000000 
true root           :7.0000000000000000
calculated root is  :7.0000000000000000

initial guess: 8.000000 
true root           :7.5000000000000000
calculated root is  :7.5000000000000000

